# Buckaroo works in Solara too

In [ ]:
import pandas as pd
import solara
from buckaroo.solara_buckaroo import SolaraDFViewer, SolaraBuckarooWidget

In [ ]:
df = pd.DataFrame({'a':[10,20]})
@solara.component
def Page():
    bw = SolaraDFViewer(df=df)
display(Page())

## You can turn off pinned_rows too

In [ ]:
df = pd.DataFrame({'a':[10,20]})
@solara.component
def Page():
    bw = SolaraDFViewer(df=df, pinned_rows=[])
display(Page())

# And it works with Polars

In [ ]:
import polars as pl
from buckaroo.solara_buckaroo import SolaraPolarsDFViewer, SolaraPolarsBuckarooWidget
pldf = pl.DataFrame({'a':[10,20]})
@solara.component
def Page():
    bw = SolaraPolarsDFViewer(df=pldf, pinned_rows=[])
display(Page())

# Let's look at more data

In [ ]:
cb_trips_df = pd.read_csv("https://github.com/paddymul/buckaroo-data/raw/main/cb_data/2016-04.csv")

In [ ]:
@solara.component
def Page():
    bw = SolaraDFViewer(df=cb_trips_df, pinned_rows=[])
display(Page())

# Building an app around SolaraDFViewer
The following app hardcodes two dataframes and lets you select between them

In [ ]:
dataframe_names = ["foo", "bar"]
df_name = solara.reactive("foo")

dataframes = dict(foo=df,
                  bar=pd.DataFrame({'c':[20,50,90], 'd':[False, True, True]}))

@solara.component
def Page():
    solara.Select(label="df_name", value=df_name, values=dataframe_names)
    active_df = dataframes[df_name.value]
    bw = SolaraDFViewer(active_df)

    solara.Markdown(f"**Selected**: {df_name.value}")
display(Page())

# Reading large dataframes

In [ ]:
url_base = "https://github.com/paddymul/buckaroo-data/raw/main/cb_data/"

dataframe_names = [
    "2016-01.parq", "2016-02.parq",
    "2016-03.parq", "2016-04.parq"]

df_name = solara.reactive("2016-01.parq")

@solara.component
def Page():
    solara.Select(label="df_name", value=df_name, values=dataframe_names)

    @solara.lab.use_task(dependencies=[df_name.value])
    def read_df():
        return pl.read_parquet(url_base+df_name.value)

    solara.ProgressLinear(read_df.pending)
    if read_df.latest is not None:
        with solara.Div(style={"opacity": 0.3 if read_df.pending else 1.0}):
            SolaraPolarsDFViewer(read_df.latest)

    solara.Markdown(f"**Selected**: {df_name.value}")
display(Page())